In [ ]:
pip install mne

In [ ]:
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew , variation
from scipy.signal import welch
import pandas as pd
from scipy.signal import hilbert
import glob

In [ ]:
matplotlib qt

In [ ]:
import os

In [ ]:
os.chdir('C:\\Users\Siriususer\Desktop\downsampled')

In [ ]:
filenames = glob.glob('*_ASSR.vhdr')
filenames

In [ ]:
ID=[]
for f in filenames:
    ID.append(f[:8]) 
ID    

In [ ]:
EEG = []

for i in range (len(ID)):  
  name= f'{ID[i]}_ASSR.vhdr'
  E = mne.io.read_raw_brainvision(name, 
                                  eog=('HEOGL', 'HEOGR', 'VEOGb'), 
                                  misc='auto', scale=1.0, 
                                  preload=True, verbose=None
                                 )
  EEG.append(E)

In [ ]:
EEG[52].info

In [ ]:

for i in range (len(EEG)):

    EEG[i].plot(start=0, n_channels=31, scalings='0.0001')

In [ ]:
#смотрим список плохих каналов
for i in range (len(EEG)):
    print(ID[i])
    print(EEG[i].info['bads'])

In [ ]:
#Интерполируем плохие каналы
for i in range (len(EEG)):
  EEG[i]= EEG[i].interpolate_bads(reset_bads=True, method=dict(eeg='MNE'), verbose=True)

In [ ]:
#удаляем каналы фотодатчика и микрофона
for i in range (len(EEG)):
  EEG[i]=EEG[i].drop_channels(['Photo', 'micr'])  
 
  EEG[i].plot(start=0,scalings='0.0001')

In [ ]:
for i in range (len(EEG)):
    EEG[i]= EEG[i].set_eeg_reference(ref_channels="average")
    #EEG[i].plot(start=0, n_channels=31, scalings='0.0001')

In [ ]:
print(EEG[-1].info['ch_names'])

In [ ]:
#сохранение preprocessed файлов
for i in range (len(ID)):
  EEG[i].save('preproc'+ID[i]+ '_eeg' + '.fif', overwrite=True)

In [ ]:
#загрузка preprocessed файлов если нужно

EEG = []

for i in range (len(ID)):  
  name= f'preproc{ID[i]}_eeg.fif'
  E = mne.io.read_raw_fif (name, preload=True)
  EEG.append(E)

In [1]:
#смотрим, какие каналы плохие

for i in range (len(EEG)):
    print(ID[i])
    print(EEG[i].info['bads'])

In [ ]:
#создаём список меток
Evlist = []
for i in range (0, len(EEG)):
  events=mne.events_from_annotations(EEG[i]) 
  Evlist.append(events)

In [ ]:
Evlist[i]


In [ ]:
for i in range (0, len(EEG)):
    if 'Stimulus/S  2' in list(Evlist[i][1].keys()):
        ev_id = Evlist[i][1]['Stimulus/S  2']
    else: ev_id = Evlist[i][1]['Stimulus/S2']
    print(ev_id)   

In [ ]:
#делим на эпохи
eplist = []
baseline=(-0.2,0)
threshold=350e-6
for i in range (0, len(EEG)):
    if 'Stimulus/S  2' in list(Evlist[i][1].keys()):
        ev_id = Evlist[i][1]['Stimulus/S  2']
    else: ev_id = Evlist[i][1]['Stimulus/S2']
    epochs_all = mne.Epochs(EEG[i], Evlist[i][0], event_id=[ev_id], # Коды меток, которые будут взяты в анализ
                                        tmin=-0.2, tmax=0.8, preload=True, # Временной интервал вокруг события, который будет взят для анализа
                                        baseline=baseline, 
                                        reject=dict(eeg=threshold)
                           )
    eplist.append(epochs_all)

In [ ]:
#график меток и массив с метками
fig = mne.viz.plot_events(Evlist[-1][0])
Evlist[-1][0][:10]

In [ ]:
#какие коды меток у нас есть?
for i in range (0, len(EEG)):
    codes=list(eplist[i].event_id.values())
    print(codes)

In [ ]:
#какие у нас эпохи и сколько
for i in range (len(EEG)):
    print(ID[i])
    print(eplist[i])

In [ ]:
#определение функции обрезки по стандартному отклонению (5std)
def dpop_epochs_by_sigma(epochs_1):

    data_epochs_1=epochs_1.get_data()
    mean_=np.mean(data_epochs_1,(0,2))
    std_=np.std(data_epochs_1,(0,2))

    max_=(mean_+5*std_)
    min_=(mean_-5*std_)


    axx=data_epochs_1.shape[0]
    axx2=data_epochs_1.shape[2]
    rej=[]

    for i in range(axx):
      for j in range(axx2):
        if np.sum(max_-data_epochs_1[i,:,j]<0)!=0 or np.sum(min_-data_epochs_1[i,:,j]>0)!=0:
                rej.append(i)

        
    reject_list=np.unique(rej).tolist()
    epochs_1.drop(reject_list)
    return rej

In [ ]:
#обрезка по стандартному отклонению
for i in range(len(eplist)):
    dpop_epochs_by_sigma(eplist[i])

In [ ]:
#усреднение по эпохам внутри испытуемых

sub_epochs=[]

for i in range (len(EEG)): 
    avg_epochs=eplist[i].average()
    sub_epochs.append(avg_epochs)

In [ ]:
for i in range(len(sub_epochs)): #отрисовать SW для каждого испытуемого
    sub_epochs[i].plot(window_title= 'SW', picks='F7')

In [ ]:
SW=[]

SW_=mne.grand_average(sub_epochs)
SW.append(SW_)

In [ ]:
SW_.plot()

In [ ]:
for i in range (len(EEG)):
    print(ID[i])
    print(sub_epochs[i])

In [ ]:
type(avg_epochs)


In [ ]:
mean_amp_roi_list, IDs = [], []

for i in range (len(EEG)):

# Select all of the channels and crop to the time window
    channels = ["F7"]
    sub_epochs_mean_roi = sub_epochs[i].copy().pick(channels).crop(tmin=0.25, tmax=0.55)

# Extract mean amplitude in µV over time
    mean_amp_roi_list.append(sub_epochs_mean_roi.data.mean(axis=1)[0] * 1e6)
    
    IDs.append(ID[i])
    

# Store the data in a data frame
mean_amp_roi_df = pd.DataFrame(
    {
#        "ch_name": sub_epochs_mean_roi.ch_names,
        "ID": IDs,
        "mean_amp": mean_amp_roi_list,
    }
)
# Print the data frame
#print(mean_amp_roi_df[i].mean(numeric_only=True))

In [ ]:
mean_amp_roi_df